In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from scipy import sparse as ssp
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")
import time

In [2]:
Path = "C:\\Users\\Himanshu\\Downloads\\train_aox2Jxw\\"
train_file = "train.csv"
test_file ="test_bqCt9Pv.csv"

train_df = pd.read_csv(Path+train_file)
test_df = pd.read_csv("C:\\Users\\Himanshu\\Downloads\\"+test_file)
display(train_df.shape)
test_df['loan_default']=0
data = pd.concat([train_df, test_df])
print(data.shape)
data.drop(columns= ['loan_default'], inplace= True)
display(data.shape)

(233154, 41)

(345546, 41)


(345546, 40)

In [3]:
""" Function to convert a Yrs b months to months"""
def total_months(x):
    l =[]
    l.append(x.split(' '))
    return int(l[0][0][0])*12 + int(l[0][1][0])

def variable_cleaning(data):
    
    """Taking logarithmic transformations of variables to make the algorithm converge """
  
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('No Bureau History Available', 0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: Sufficient History Not Available', 0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: Not Enough Info available on the customer', 0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: No Activity seen on the customer (Inactive)',0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: No Updates available in last 36 months', 0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: Only a Guarantor', 0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: More than 50 active Accounts found',0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('M-Very High Risk', 1)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('L-Very High Risk', 1)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('K-High Risk', 2)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('J-High Risk', 2)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('I-Medium Risk', 3)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('H-Medium Risk', 3)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('G-Low Risk', 4)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('F-Low Risk', 4)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('E-Low Risk', 4)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('D-Very Low Risk', 5)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('C-Very Low Risk', 5)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('B-Very Low Risk', 5)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('A-Very Low Risk', 5)
    
        
    data['Converted_AVERAGE.ACCT.AGE'] = train_df['AVERAGE.ACCT.AGE'].apply(total_months)
    data['Converted_CREDIT.HISTORY.LENGTH'] = train_df['CREDIT.HISTORY.LENGTH'].apply(total_months)
    
    
    data['Employment_ind'] = data['Employment.Type'].apply(lambda x: (1 if x == 'Self employed' else (0 if x == 'Salaried' else -1)))

    return data


In [4]:

def creating_direct_variables(train_df, test_df):
    
    # 1. Age at loan disbursal date

    train_df['Date.of.Birth'] = pd.to_datetime(train_df['Date.of.Birth'].values)
    train_df['DisbursalDate'] = pd.to_datetime(train_df['DisbursalDate'].values)
    train_df['Age_at_disbursaldate'] = (train_df['DisbursalDate'].dt.date - train_df['Date.of.Birth'].dt.date)/ np.timedelta64(365, 'D')

    test_df['Date.of.Birth'] = pd.to_datetime(test_df['Date.of.Birth'].values)
    test_df['DisbursalDate'] = pd.to_datetime(test_df['DisbursalDate'].values)
    test_df['Age_at_disbursaldate'] = (test_df['DisbursalDate'].dt.date - test_df['Date.of.Birth'].dt.date)/ np.timedelta64(365, 'D')
    
    # 2. Total number of verified identities

    train_df['Total_verification_flags'] = (train_df['Aadhar_flag'] + train_df['PAN_flag']+ train_df['VoterID_flag'] \
                                        +train_df['Driving_flag'] + train_df['Passport_flag'])
    
    test_df['Total_verification_flags'] = (test_df['Aadhar_flag'] + test_df['PAN_flag']+ test_df['VoterID_flag'] \
                                        +test_df['Driving_flag'] + test_df['Passport_flag'])

    # 3. Disbursal Date and Month
    
    train_df['Disbursal_Month'] = train_df['DisbursalDate'].dt.month
    train_df['Disbursal_Day'] = train_df['DisbursalDate'].dt.day

    test_df['Disbursal_Month'] = test_df['DisbursalDate'].dt.month
    test_df['Disbursal_Day'] = test_df['DisbursalDate'].dt.day 
    
    # 4. PAn and Voter Id combo
    
    train_df.loc[((train_df['VoterID_flag'] == 0) & (train_df['PAN_flag'] == 0)),'VoterId_Pan_flag_Combo'] = 1
    train_df.loc[((train_df['VoterID_flag'] == 0) & (train_df['PAN_flag'] == 1)),'VoterId_Pan_flag_Combo'] = 2
    train_df.loc[((train_df['VoterID_flag'] == 1) & (train_df['PAN_flag'] == 0)),'VoterId_Pan_flag_Combo'] = 3
    train_df.loc[((train_df['VoterID_flag'] == 1) & (train_df['PAN_flag'] == 1)),'VoterId_Pan_flag_Combo'] = 4


    test_df.loc[((test_df['VoterID_flag'] == 0) & (test_df['PAN_flag'] == 0)),'VoterId_Pan_flag_Combo'] = 1
    test_df.loc[((test_df['VoterID_flag'] == 0) & (test_df['PAN_flag'] == 1)),'VoterId_Pan_flag_Combo'] = 2
    test_df.loc[((test_df['VoterID_flag'] == 1) & (test_df['PAN_flag'] == 0)),'VoterId_Pan_flag_Combo'] = 3
    test_df.loc[((test_df['VoterID_flag'] == 1) & (test_df['PAN_flag'] == 1)),'VoterId_Pan_flag_Combo'] = 4

    
    return train_df, test_df

In [17]:
def creating_dfg_variables(train_df,val,test_df):
    # 5. Creating default to gross variables
    
    
    df_supplier_id_ftg      = pd.DataFrame(train_df.groupby('supplier_id')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0)).rename(columns = {'loan_default': 'supplier_id_ftg'})
    df_branch_id_ftg        = pd.DataFrame(train_df.groupby('branch_id')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0)).rename(columns = {'loan_default': 'branch_id_ftg'})
    df_manufacturer_id_ftg  = pd.DataFrame(train_df.groupby('manufacturer_id')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0)).rename(columns = {'loan_default': 'manufacturer_id_ftg'})
    df_pincode_id_ftg       = pd.DataFrame(train_df.groupby('Current_pincode_ID')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20  else 0)).rename(columns = {'loan_default': 'pincode_id_ftg'})
    df_state_id_ftg         = pd.DataFrame(train_df.groupby('State_ID')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20  else 0)).rename(columns = {'loan_default': 'state_id_ftg'})
    df_employee_code_id_ftg = pd.DataFrame(train_df.groupby('Employee_code_ID')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0)).rename(columns = {'loan_default': 'employee_code_id_ftg'})
    df_combo_state_pin_ftg = pd.DataFrame(train_df.groupby(['State_ID','Current_pincode_ID'])['loan_default']\
                                          .apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0))\
                                            .rename(columns = {'loan_default': 'state_pin_code_ftg'})
    
    train_df['supplier_id_ftg'] = df_supplier_id_ftg.loc[train_df.supplier_id ,'supplier_id_ftg'].values
    train_df['branch_id_ftg'] = df_branch_id_ftg.loc[train_df.branch_id ,'branch_id_ftg'].values
    train_df['manufacturer_id_ftg'] = df_manufacturer_id_ftg.loc[train_df.manufacturer_id ,'manufacturer_id_ftg'].values
    train_df['pincode_id_ftg'] = df_pincode_id_ftg.loc[train_df.Current_pincode_ID ,'pincode_id_ftg'].values
    train_df['state_id_ftg'] = df_state_id_ftg.loc[train_df.State_ID ,'state_id_ftg'].values
    train_df['employee_code_id_ftg'] = df_employee_code_id_ftg.loc[train_df.Employee_code_ID ,'employee_code_id_ftg'].values
    train_df['state_pin_code_ftg'] = df_combo_state_pin_ftg.loc[ (train_df.State_ID == df_combo_state_pin_ftg.index.get_level_values(0)) & \
                            (train_df.Current_pincode_ID == df_combo_state_pin_ftg.index.get_level_values(1)),'state_pin_code_ftg'].values

    val['supplier_id_ftg'] = df_supplier_id_ftg.loc[val.supplier_id ,'supplier_id_ftg'].values
    val['branch_id_ftg'] = df_branch_id_ftg.loc[val.branch_id ,'branch_id_ftg'].values
    val['manufacturer_id_ftg'] = df_manufacturer_id_ftg.loc[val.manufacturer_id ,'manufacturer_id_ftg'].values
    val['pincode_id_ftg'] = df_pincode_id_ftg.loc[val.Current_pincode_ID ,'pincode_id_ftg'].values
    val['state_id_ftg'] = df_state_id_ftg.loc[val.State_ID ,'state_id_ftg'].values
    val['employee_code_id_ftg'] = df_employee_code_id_ftg.loc[val.Employee_code_ID ,'employee_code_id_ftg'].values
    val['state_pin_code_ftg'] = df_combo_state_pin_ftg.loc[(val.State_ID == \
                                  df_combo_state_pin_ftg.index.get_level_values(0)) &\
                            (val.Current_pincode_ID == df_combo_state_pin_ftg.index.get_level_values(1)),'state_pin_code_ftg'].values


    test_df['supplier_id_ftg'] = df_supplier_id_ftg.loc[test_df.supplier_id ,'supplier_id_ftg'].values
    test_df['branch_id_ftg'] = df_branch_id_ftg.loc[test_df.branch_id ,'branch_id_ftg'].values
    test_df['manufacturer_id_ftg'] = df_manufacturer_id_ftg.loc[test_df.manufacturer_id ,'manufacturer_id_ftg'].values
    test_df['pincode_id_ftg'] = df_pincode_id_ftg.loc[test_df.Current_pincode_ID ,'pincode_id_ftg'].values
    test_df['state_id_ftg'] = df_state_id_ftg.loc[test_df.State_ID ,'state_id_ftg'].values
    test_df['employee_code_id_ftg'] = df_employee_code_id_ftg.loc[test_df.Employee_code_ID ,'employee_code_id_ftg'].values
    test_df['state_pin_code_ftg'] = df_combo_state_pin_ftg.loc[(test_df.State_ID == \
                                  df_combo_state_pin_ftg.index.get_level_values(0)) &\
                            (test_df.Current_pincode_ID == df_combo_state_pin_ftg.index.get_level_values(1)),'state_pin_code_ftg'].values


    
    return train_df,val,test_df

In [6]:
param = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.01,
    'num_leaves': 255,  
    'max_depth': 7,  
    'min_child_samples':50,  
    'max_bin': 100,  
    'subsample': 0.7,  
    'subsample_freq': 1,  
    'colsample_bytree': 0.7,  
    'min_child_weight': 0,  
    'subsample_for_bin': 200,  
    'min_split_gain': 0,  
    'reg_alpha': 1,  
    'reg_lambda': 1,  
   # 'nthread': 8,
    'verbose': 5,
    'scale_pos_weight':99 
    }
        

In [7]:
train_df = variable_cleaning(train_df)
test_df = variable_cleaning(test_df)

train_df, test_df = creating_direct_variables(train_df, test_df)

In [18]:
# Doing K fold crossvalidation

NFOLDS = 5
folds = StratifiedKFold(n_splits= NFOLDS, shuffle=True, random_state=218)
all_f = train_df.columns
features = ['disbursed_amount', 'asset_cost', 'ltv', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',\
                'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS',
                'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
                'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
               'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
               'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
               'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'NO.OF_INQUIRIES',
               'Age_at_disbursaldate',   'Converted_AVERAGE.ACCT.AGE',
               'Converted_CREDIT.HISTORY.LENGTH', 'Employment_ind','Total_verification_flags','VoterId_Pan_flag_Combo',
           'Disbursal_Month','Disbursal_Day','PERFORM_CNS.SCORE.DESCRIPTION', 'supplier_id_ftg',
            'branch_id_ftg','pincode_id_ftg', 'manufacturer_id_ftg',  'state_id_ftg','state_pin_code_ftg']
#  'employee_code_id_ftg'

categorical_columns = ['VoterId_Pan_flag_Combo','Disbursal_Month','Employment_ind']
predictions1 = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()
start_time= time.time()
score = [0 for _ in range(folds.n_splits)]
oof = np.zeros(len(train_df))
max_iter = 5
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, train_df['loan_default'].values)):
    print("fold n : {}".format(fold_))
    

  
    train,val, test_df = creating_dfg_variables(train_df.iloc[trn_idx], train_df.iloc[val_idx], test_df)
    
    trn_data = lgb.Dataset(train[features],
                           label=train_df.iloc[trn_idx]['loan_default'], categorical_feature = categorical_columns
                          )
    val_data = lgb.Dataset(val[features],
                           label=train_df.iloc[val_idx]['loan_default'] , categorical_feature = categorical_columns
                          )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 100)
    print(val_idx)
    print(len(oof))
    print(len(val_idx))
    oof[val_idx] = clf.predict(val[features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    # we perform predictions by chunks
    current_pred = clf.predict(test_df[features].values)
    predictions1 += current_pred / min(folds.n_splits, max_iter)
   
    print("time elapsed: {:<5.2}min".format((time.time() - start_time) / 60))
    score[fold_] = roc_auc_score(train_df.iloc[val_idx]['loan_default'].values, oof[val_idx])
    if fold_ == max_iter - 1: break
        
if (folds.n_splits == max_iter):
    print("CV score: {:<8.5f}".format(metrics.roc_auc_score(train_df['loan_default'].values, oof)))
else:
     print("CV score: {:<8.5f}".format(sum(score) / max_iter))


fold n : 0


ValueError: Lengths must match to compare

In [10]:

# print(roc_auc_score(Y_test, final_scores))
submission = pd.read_csv("C:\\Users\\Himanshu\\Downloads\\"+test_file)
submission['loan_default'] = predictions1
submission[['UniqueID','loan_default']].to_csv('D3_3nd_solution.csv', index = False)

In [26]:
!pip install xgboost

In [11]:
df_combo_state_pin_ftg = pd.DataFrame(train_df.groupby(['State_ID','Current_pincode_ID'])['loan_default']\
                                          .apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0))\
                                            .rename(columns = {'loan_default': 'employee_code_id_ftg'})

In [14]:
df_combo_state_pin_ftg

employee_code_id_ftg
State_ID Current_pincode_ID                      
1        1                               0.230769
         2                               0.180556
         3                               0.120000
         4                               0.170455
         5                               0.269767
         6                               0.220000
         7                               0.163462
         8                               0.209302
         9                               0.137931
         10                              0.000000
         11                              0.090909
         12                              0.100000
         13                              0.155556
         14                              0.168317
         15                              0.238095
         16                              0.122449
         17                              0.141593
         18                              0.210938
         19                              0.163636
         20                              0.000000
         21                              0.103448
         22                              0.000000
         23                              0.000000
         24                              0.142857
         25                              0.000000
         26                              0.125000
         27                              0.096774
         28                              0.000000
         29                              0.208333
         30                              0.193966
...                                           ...
20       1552                            0.176471
         1553                            0.000000
         1554                            0.130435
21       2913                            0.000000
         2914                            0.000000
         2916                            0.000000
         2917                            0.000000
         2919                            0.375000
         2920                            0.000000
         2921                            0.000000
         2922                            0.000000
         2923                            0.000000
         2925                            0.000000
         2927                            0.140351
         2928                            0.000000
         2929                            0.000000
         2930                            0.000000
         2931                            0.000000
         2932                            0.000000
         2933                            0.000000
         2934                            0.000000
         2935                            0.000000
         2938                            0.000000
22       544                             0.142857
         545                             0.000000
         546                             0.000000
         547                             0.000000
         548                             0.071429
         549                             0.000000
         550                             0.000000

[6698 rows x 1 columns]

In [ ]:
def Stacking(model,train,y,test,n_fold):
    folds=StratifiedKFold(n_splits=n_fold,random_state=1)
    test_pred=np.empty((test.shape[0],1),float)
    train_pred=np.empty((0,1),float)
    for train_indices,val_indices in folds.split(train,y.values):
        x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
        y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

        model.fit(X=x_train,y=y_train)
        train_pred=np.append(train_pred,model.predict(x_val))
        test_pred=np.append(test_pred,model.predict(test))
    return test_pred.reshape(-1,1),train_pred
